### Tema "Kvinde og dannelse" med ordsekker
Kode utviklet av Lars G.Johnsen

#### Boot cell

Skip if `module_update.py` is in same folder as notebook, next cell downloads this file which in turn will download other files.

In [69]:
!curl "https://raw.githubusercontent.com/Yoonsen/Modules/master/module_update.py" > "module_update.py"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3823  100  3823    0     0  13349      0 --:--:-- --:--:-- --:--:-- 14211


#### Get essentials

Import text and graph modules for analysis, and css for display

In [70]:
from module_update import update, css, printmd

In [71]:
css()

# Generisk analyse

Intro

### Start med å importere modulene

In [72]:
update('nbtext', overwrite=True)
update('graph_networkx_louvain')
import nbtext as nb
#import graph_networkx_louvain as gnl
from graph_networkx_louvain import cutdown, make_collocation_graph 

Updated file `/Users/heidikarlsen/Documents/Doktorgrad/Jupyter Notebook/nbtext.py`

Updated file `/Users/heidikarlsen/Documents/Doktorgrad/Jupyter Notebook/graph_networkx_louvain.py`

In [73]:
def link(urn):
    return "https://www.nb.no/items/URN:NBN:no-nb_digibok_{u}".format(u = urn)

Her er en uvektet jaccard som sjekker de radene som har mer enn ett visst antall treff, bestemt av parameteret `limit`.

In [47]:
def interesting_urns(df, limit):
    jaccard = dict((df > 0).astype(int).sum(axis=1))
    res = []
    for urn in jaccard:
        if jaccard[urn] >= limit:
            res.append(urn)
    return res

In [48]:
def get_para(para_list):
    import ast
    paras = []
    for p in para_list:
        paras.append(ast.literal_eval(p))
    urns = [x[0] for x in paras]
    paradict = {u:[x[1] for x in paras if x[0] == u] for u in urns}
    paragraphs = dict()
    for u in paradict:
        paragraphs[u] = get_paragraphs(u, paradict[u])
    return paragraphs

In [49]:
def interesting_paragraphs(urns, wordbag, limit):
    interest = dict()
    for u in urns:
        paragraphs = nb.wordbag_eval_para(wordbag, [u])
        for x in paragraphs:
            if len(paragraphs[x]) >= limit:
                interest[x] = paragraphs[x]
    return interest

In [50]:
import requests
def get_paragraphs(urn, paras):
    """Return paragraphs for urn"""
    param = dict()
    param['paragraphs'] = paras
    param['urn'] = urn
    r = requests.get("https://api.nb.no/ngram/paragraphs", json=param)
    return r.json()

## Bygg korpus og analysér

Hent URN-er fra tekst eller metadata

### Hent ut bøker fra 1830 - 1880

In [51]:
urns =nb.get_urn({'year': 1830, 'next':50, 'limit':10000}) 

In [52]:
len(urns)

6803

## Ordsekker

Definer hyperleksemer som ordsekker. Definisjonen er på formen 
"""definisjon : ord1, ord2, ..., ordN;"""
Kommandoen `def2dict` konverterer definisjonen til en pythonstruktur som kan brukes videre. Den vil samtidig legge til en kapitalisert variant av alle ord som ikke har en.

In [53]:
Kvinde_dannelse = nb.def2dict(
"""Kvinde_: kvinne, kvinnen, kvinner, kvinnene, kvinnerne, kvinnes, kvinnens, kvinners, kvinnenes, kvinnernes, kvinde, kvinden, kvinder, kvinderne, kvindens, kvindes, kvindernes, kvinders, kvindelige, kvindelig, kvindeligt, kvindene, kvindeme, kvindekøn, kvindenes, kvindemes, kvindc, kvindcr, kvindcn, kvindcns, kvindcs, kvindcrnes, kvindcrnc, kvindcrs, kvindcrncs, kvindclighed, kvindcligheden, kuindc, kuindcr, kuindcn, kuindcs, kuindcrne, kuindcns, kuinde, kuinder, kuinden, kuinderne, kuindens, kuinders, kuindes, kvindelighed, kvindeligt, kvindelighedens, kvindeligheden, kvindeligheds, kvindelighed, kvindclige, kvindelighcd, kvindeligked, kvindeligbed, qvinde, qvinden, qvinder, qvinderne, qvindene, qvindelig, qvindelige, qvindelighed, quinde, quinden, quinder, qvinderne, qvindene, quindelig, quindelighed, qvindcn, quindcn, qvindclig, quindclig, qvindcr, quindcr, qvindcrne, qvindene, quindcrne, quindcne, qvindclighcd, quindclighcd, qvindelighcd, quindelighcd, qvindclighed, quindclighcd, Dame, Damen, dame, Damen, damc, Damc, damcn, Damcn, damer, Damer, damcr, Damcr, damene, Damene, damcne, Damcne, damcrne, Damcrne, damerne, Damerne, damcrnc, Damcrnc, damcnc, Damcnc, damernc, Damernc, damenc, Damenc, jente, jenten, jenta, jenter, jentene, jenterne, jentas, jentens, jentes, jenters, jentenes, jenterne, jenternes, jcnte, jcnten, jcnterne, pike, piken, piker, pikene, pikerne, pikes, pikens, pikers, pikenes, pikernes, pikebarn, pikebørn, pige, piger, pigene, pigerne, piges, pigens, pigers, pigenes, pigernes, pigebarn, pigebarnet, pigeborn, pigc, pigcn, pigcr, pigcne, pigcrne, pigcs, pigcbarn, pigcbarnet, pigcborn, heltinne, heltinnen, heltinner, heltinnene, heltinnens, hcltinnen, heltinde, heltinden, heltinder, heltindene, heltinderne, heltindens, heltindc, tjenerinne, tjenerinnen, tjenerinde, tjenerinden, tjcnerinde, tjcncrinde, tjcnerinden, tjcncrinden, tjcncrindcn, fristerinne, fristerinnen, fristerinnens, fristerinde, fristerinden, fristerindens, fristcrinde, fristcrinden, fristcrindcn, kone, konen, koncn, hustru, huustru, fruentimmer, fruentimmeret, fruentimmere, fruentimmerne, fruentimmernc, fruentimmers, fruentimmerets, fruentimmeraktig, fruentimmeragtig, fruentimmerlig, fruentimmerlige, fruentimmerfølelser, fruentimre, fruentimret, fruentimrene, qvindekjønnet, qvindekjonnet, qvindckjonnet, quindckjonnet, quindekjønnet, quindekjonnet, quindckjonnet, qvindekjonn, qvindckjonn, quindekjonn, quindckjonn, beskjørtede, bcskjørtede, bcskjørtcde, bcskjortcde, beskjortcde, bcskjortede, beskjortede, fruentimmernetheder, fruentimmcrncdhcer, frucntimmcrncdhcdcr, husmødrene, husmodrene, husmoder, husmor, husmodrcne, mor, moder, moderlig, modre, modrcne, modrc, modcrlig, modcrlighcden, modcrlighed, moderlighed, modcrlighcd, telegrafistinde, telegrafistinder, telegrafistinderne, telcgrafistinde, tclcgrafistinde, tclcgrafistindcr, telcgrafistindcrne, lcererinde, lærerinde, lcercrinde, lcercrinder, lcererinderne, lcercrindene, mor, moder, moderlig, modre, modrcne, modrc, modcrlig, modcrlighcden, modcrlighed, moderlighed, modcrlighcd, telegrafistinde, telegrafistinder, telegrafistinderne, telcgrafistinde, tclcgrafistinde, tclcgrafistindcr, telcgrafistindcrne, lcererinde, lærerinde, lcercrinde, lcercrinder, lcererinderne, lcercrindene, Huusholderske, Huusholdersken, Huusmoder, husmoderlig, husmoderlige, husmoderlighed, husmoderligheden, Husmor, Husmoder, Husmodcr, Husmodcrcn, Husmoderen, Husmodcren, Husmodcren, Huusmodcr, Huusmodre, Huusmoders, Huusmodcr, Huusmodrene, Huusmodcren, Huusmodercn, Huusmodcrs, Husmodre, Husmodrc, Husmoderhjerne, Husmoderkall, Husmoderem, Husmoderms, Huusmodcrcn, Huusholderffe, Huusholderster, Huusholderste, Husfrue, Huusfrue, Huusfrne, Favoritinde, Husslavinde, Emancipationsistinde, Emancipationistindern, Læserinder, Læserinde, Lceserinde, Lceserinder, Læserinden, Leserinder, Leserinde, Læserinders, Lcrserinde; 
Dannelse_: Dannelse, Dannelsens, Dannet, Uddannelse, Uddannelsen, Uddanning, Uddannclse, Uddannclsen, Uddannclscn, Dannclse, Dannclsen, Danclsens, Dannct, Dannclsc, Dannclscn, Danncr, Danncde, Danncs, Danncdc, Danncnde, Utvikling, Utviklingen, Utviklingens, Utviklings, Utvikle, Utvikler, Utvikles, Utviklict, Utviklcr, Utviklcs, Utciklcde, Oplæring, Oplæringen, Oplæringens, Oplærings, Mål, Målet, Målct, Maal, Maalet, Maalct, Formål, Formålet, Formålcne, Formaal, Formaalet, Formaalets, Formaals, Fomaalenes, Kunskab, kunskaben, kunskaber, kunskabsrig, Kundskab, kundskaben, kundskaber, Kundskabsrig, Kunnskab, kunnskaben, kunnskaber, Kunnskabsrig, Kunnskabsrik, Kundskabsrik, Kunskabsrik,  kunskabs, kunnskabens, kundskabere, kundskabssyn, kundskabelig, kunskabere, kundskabss, kunclskaber, Visdom, Visdommen, Visdommens, Visdommcn, Visdommcns, Lcerdom, Lcrrdom, Lærdom, Lerdom, Loerdom, Lardom, Laerdom, Lccrdom, Lwrdom, Leerdom, Lsrdom, Lerrdom, Lserdom, Llrrdom, Lnrdom, Lierdom, Locrdom, Ltrrdom, Lssrdom, LErdom, LcerdoM, LcrrdoM, Llerdom, Lcvrdom, Lecrdom, Aandsdannelse, Aandsdannelsen, Aandsdannelsens, Aandsdannclse, Aandsdannelses, Aandsdannelsc, Aandsdannelst, Aandsdannclsc, AandSdannelse; 
Sjel_: Sjel, Sjelen, Sjels, Sjelens, Sjelelig, Sjelelige, Sjcl, Sjclen, Sjcls, Sjclens, Sjclcn, Sjclcns, Sjclelig, Sjclelige, Kvinnesjel, Kvinnesjelen, Kvinnesjelens, Kvindesjel, Kvindesjelen, Kvindesjelens, Kvindesjæl, Kvindesjælen, Kvindesjælens, Kvindesjæls, Aand, aandelig, Aandsliv, aandsrik, aandfuld, aandclig, aandeli, aandélig Aandelighed, Aandeligt, Aandclige, Aandeliges, Aandeligheds, Aandeligc, Aandclig, Aandcligc, Aandclighed, Aandtlige, Aandcligt, Aandklige, Aandelighcd, vis, vise, visc, intelligens, intelligence, inteligens, inteligent, intelligent, begaved, begavelse, aandsevner, aandsevnerne, Bevidsthed, Tankeliv;"""
)
print(Kvinde_dannelse)

{'Kvinde_': ['kvinne', 'kvinnen', 'kvinner', 'kvinnene', 'kvinnerne', 'kvinnes', 'kvinnens', 'kvinners', 'kvinnenes', 'kvinnernes', 'kvinde', 'kvinden', 'kvinder', 'kvinderne', 'kvindens', 'kvindes', 'kvindernes', 'kvinders', 'kvindelige', 'kvindelig', 'kvindeligt', 'kvindene', 'kvindeme', 'kvindekøn', 'kvindenes', 'kvindemes', 'kvindc', 'kvindcr', 'kvindcn', 'kvindcns', 'kvindcs', 'kvindcrnes', 'kvindcrnc', 'kvindcrs', 'kvindcrncs', 'kvindclighed', 'kvindcligheden', 'kuindc', 'kuindcr', 'kuindcn', 'kuindcs', 'kuindcrne', 'kuindcns', 'kuinde', 'kuinder', 'kuinden', 'kuinderne', 'kuindens', 'kuinders', 'kuindes', 'kvindelighed', 'kvindeligt', 'kvindelighedens', 'kvindeligheden', 'kvindeligheds', 'kvindelighed', 'kvindclige', 'kvindelighcd', 'kvindeligked', 'kvindeligbed', 'qvinde', 'qvinden', 'qvinder', 'qvinderne', 'qvindene', 'qvindelig', 'qvindelige', 'qvindelighed', 'quinde', 'quinden', 'quinder', 'qvinderne', 'qvindene', 'quindelig', 'quindelighed', 'qvindcn', 'quindcn', 'qvindclig

Kommandoen `nb.wordbag()` tar en orddefinisjon og en liste URN-er og søker i dem for en første sjekk. Det søket fungerer som en grov hash-index for avsnittene som sjekkes under.

In [54]:
df = nb.wordbag_eval(Kvinde_dannelse, urns[:1000])

In [55]:
df2 = nb.wordbag_eval(Kvinde_dannelse, [x[0] for x in urns[1000:2000]])

In [56]:
totdf = df.append(df2).sort_values(by='snitt', ascending=False)
totdf.head(20)

,Dannelse_,Kvinde_,Sjel_,snitt
2009091403021,319.0,1439.0,697.0,818.333333
2008082512001,251.0,1183.0,680.0,704.666667
2014062048060,166.0,1147.0,614.0,642.333333
2016022548049,129.0,1175.0,550.0,618.000000
2010011903029,244.0,890.0,716.0,616.666667
2015010648111,564.0,476.0,492.0,510.666667
2012102924020,108.0,1160.0,243.0,503.666667
2012102924019,104.0,1117.0,245.0,488.666667
2016100729002,388.0,126.0,951.0,488.333333
2012101624017,115.0,1079.0,233.0,475.666667


## Jaccard


Interessante URNer er de som treffer med temaordene. Juster antall etter behov.

In [57]:
urns_of_interest = interesting_urns(df[list(Kvinde_dannelse.keys())], 3)
urns_of_interest

['2009091403021',
 '2008082512001',
 '2012102924019',
 '2016100729002',
 '2015010648112',
 '2015062608033',
 '2008090903023',
 '2012102924021',
 '2011060704053',
 '2009092303042',
 '2014082608042',
 '2015010648109',
 '2014112108104',
 '2010021803052',
 '2010020503005',
 '2016063008035',
 '2016072026006',
 '2008111103033',
 '2016031548121',
 '2010012603001',
 '2014061108162',
 '2009071303012',
 '2009021812004',
 '2008111903003',
 '2012080908032',
 '2008100703004',
 '2008050610001',
 '2009081303006',
 '2006112201009',
 '2007082302001',
 '2009082700002',
 '2009031312001',
 '2008112003011',
 '2008013010001',
 '2014031808023',
 '2009080603010',
 '2012042713002',
 '2010021903014',
 '2015061048067',
 '2008092912003',
 '2006083100002',
 '2009092203003',
 '2015060948119',
 '2014111908037',
 '2008092203009',
 '2010021603029',
 '2014032824001',
 '2016041548026',
 '2008111303024',
 '2012020208032',
 '2008011112004',
 '2010020303009',
 '2016101848035',
 '2008090303003',
 '2009092303060',
 '20090714

Slik ser datarammen ut for de mest interessante

In [58]:
df.loc[urns_of_interest]

,Dannelse_,Kvinde_,Sjel_,snitt
2009091403021,319.0,1439.0,697.0,818.333333
2008082512001,251.0,1183.0,680.0,704.666667
2012102924019,104.0,1117.0,245.0,488.666667
2016100729002,388.0,126.0,951.0,488.333333
2015010648112,401.0,437.0,417.0,418.333333
2015062608033,358.0,256.0,563.0,392.333333
2008090903023,300.0,137.0,734.0,390.333333
2012102924021,109.0,913.0,143.0,388.333333
2011060704053,165.0,212.0,758.0,378.333333
2009092303042,156.0,218.0,726.0,366.666667


Det kutter betydelig ned på URNene. Og de fores inn i avsnittsevaluatoren

# URNs of interest

Variabelen `urns_of_interest` ble definert over, men plassen som fylles kan være hvilken som helst bok. Så det er mulig å hoppe over alt det som gjøres i tidligere celler, og gå rett på analysen av bøker som allerede er av interesse, som Collett eller Skram, eller.

In [59]:
para_of_interest = interesting_paragraphs(urns_of_interest[:1], Kvinde_dannelse, 3)

In [60]:
para_of_interest

{"('2009091403021', 1163)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 123)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 1297)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 1330)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 1350)": {'Dannelse_': 1, 'Kvinde_': 8, 'Sjel_': 2},
 "('2009091403021', 2317)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 2972)": {'Dannelse_': 1, 'Kvinde_': 5, 'Sjel_': 2},
 "('2009091403021', 3083)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 3},
 "('2009091403021', 3145)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 4},
 "('2009091403021', 3241)": {'Dannelse_': 1, 'Kvinde_': 5, 'Sjel_': 2},
 "('2009091403021', 3439)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 3986)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 4179)": {'Dannelse_': 1, 'Kvinde_': 1, 'Sjel_': 1},
 "('2009091403021', 4239)": {'Dannelse_': 2, 'Kvinde_': 3, 'Sjel_

Her henter vi ut de paragrafene som har nok treff.

In [61]:
paras = get_para(para_of_interest)

Noen linjer for å lage ord fra temadefinisjon

In [63]:
wordhits = []
for ws in Kvinde_dannelse.values():
    wordhits += ws


In [64]:
def para2html(paras, wordhits):
    from IPython.display import HTML
    html = ""
    for urn in paras:
        metadata = nb.metadata(urn)[0]
        item = "<h4><a href='{lnk}' target='_blank'>{meta}</a></h4>".format(
            lnk = link(urn),
            meta = ', '.join([str(x) for x in metadata]))
        content = ""
        for p in paras[urn]:
            q = paras[urn][p]
            for word in wordhits:
                q = q.replace(word, "<mark>"+word+"</mark>")
            content += "<p>{paragraph}</p>".format(paragraph = q)
        html += item + content
    return HTML("<body>" + html + "</body>")

In [65]:
para2html(paras, wordhits)